In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
conda install -c conda-forge scikit-plot

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install yfinance
!pip install investpy
!pip install pykrx
!pip install seaborn
!pip install workalendar
!pip install --upgrade finance-datareader
!pip install TA_Lib-0.4.19-cp38-cp38-win_amd64.whl
!pip install -U finance-datareader
!pip install keras

Processing c:\users\user\downloads\ta_lib-0.4.19-cp38-cp38-win_amd64.whl
TA-Lib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
import yfinance as yf # yahoo finance API    # pip install yfinance
import investpy # investing.com API          # pip install investpy
from pykrx import stock # krx API           # pip instasll pykrx
import talib as ta # 기술적 분석 (보조지표)
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
%matplotlib inline

In [5]:
# 크롤링 start_date, end_date
start_date=input('YYYY-MM-DD 형식을 지켜 입력해주세요 ex) 2018-01-01 : ')
end_date=input('YYYY-MM-DD 형식을 지켜 입력해주세요 ex) 2020-10-13 : ')

YYYY-MM-DD 형식을 지켜 입력해주세요 ex) 2018-01-01 : 2017-01-01
YYYY-MM-DD 형식을 지켜 입력해주세요 ex) 2020-10-13 : 2021-01-7


In [6]:
# 데이터 수집기간 입력
# yahoo finance 양식, ex) yyyy-mm-dd
#start_date = '2018-01-01'
#end_date = '2020-10-13'

# investing.com 양식, ex) dd/mm/yyyy
start_date_ = start_date[8:] + '/' + start_date[5:7] + '/' + start_date[:4]
end_date_ = end_date[8:] + '/' + end_date[5:7] + '/' + end_date[:4]

# krx 양식 ex) yyyymmdd
start_date__ = start_date[0:4] + start_date[5:7] + start_date[8:10]
end_date__ = end_date[0:4] + end_date[5:7] + end_date[8:10]

In [7]:
# 주요 3개국 대비 원 환율

# 달러/원
exchange_rate_usd_ = investpy.get_currency_cross_historical_data(currency_cross='USD/KRW', from_date=start_date_, to_date=end_date_)
exchange_rate_usd_.columns = ['exchange_rate_usd_Open', 'exchange_rate_usd_High', 'exchange_rate_usd_Low', 'exchange_rate_usd_Close', 'exchange_rate_usd_Currency']
exchange_rate_usd_ = exchange_rate_usd_.drop(['exchange_rate_usd_Open','exchange_rate_usd_High','exchange_rate_usd_Low','exchange_rate_usd_Currency'], axis=1)

# NASDAQ
nasdaq_ = yf.download("^IXIC", start=start_date, end=end_date)
nasdaq_.columns = ['nasdaq_Open','nasdaq_High','nasdaq_Low','nasdaq_Close','nasdaq_Adj Close','nasdaq_Volume']
nasdaq_ = nasdaq_.drop(['nasdaq_Open','nasdaq_High','nasdaq_Low','nasdaq_Adj Close','nasdaq_Volume'], axis=1)

# Russell 2000
russell_2000_ = yf.download("^RUT", start=start_date, end=end_date)
russell_2000_.columns = ['russell_2000_Open','russell_2000_High','russell_2000_Low','russell_2000_Close','russell_2000_Adj Close','russell_2000_Volume']
russell_2000_ = russell_2000_.drop(['russell_2000_Open','russell_2000_High','russell_2000_Low','russell_2000_Adj Close','russell_2000_Volume'], axis=1)

# 삼성 차트 데이터
sam_ = stock.get_market_ohlcv_by_date(start_date__, end_date__, "005930")
sam_.columns = ['Open','High','Low','Close','Volume']
model_samsung = sam_.copy()

# 3) MACD 이동평균수렴확산 (단기(EMA12)와 장기(EMA26) EMA로 모멘텀을 추정)
macd, macdsignal9, macdhist = ta.MACD(model_samsung.Close, fastperiod=12, slowperiod=26, signalperiod=9)
model_samsung['macd'] = macd
model_samsung['macdsignal9'] = macdsignal9
model_samsung['macdhist'] = macdhist

# 2) 볼린저밴드 (주가의 이동평균선을 중심으로 표준편차 범위를 표시)
ubb, mbb, lbb = ta.BBANDS(model_samsung.Close, 20, 2)
model_samsung['ubb'] = ubb
model_samsung['mbb'] = mbb
model_samsung['lbb'] = lbb

# 1) 이평선(SMA, EMA, WMA) (w = 5,10,15,20,30,60,120)
model_samsung['ma_5'] = ta.SMA(model_samsung.Close, timeperiod=5)
model_samsung['ma_10'] = ta.SMA(model_samsung.Close, timeperiod=10)
model_samsung['ma_15'] = ta.SMA(model_samsung.Close, timeperiod=15)
model_samsung['ma_20'] = ta.SMA(model_samsung.Close, timeperiod=20)
model_samsung['ma_30'] = ta.SMA(model_samsung.Close, timeperiod=30)
model_samsung['ma_60'] = ta.SMA(model_samsung.Close, timeperiod=60)
model_samsung['ma_120'] = ta.SMA(model_samsung.Close, timeperiod=120)

MSCI_KR = fdr.DataReader('156080', start_date,end_date) #MSCI KOREA
MSCI_KR = MSCI_KR.drop(['Open','High','Low', 'Change','Volume'], axis=1) 
MSCI_KR.columns = ['MSCI_KR']

# SOX 지수
SOX = yf.download("^SOX", start=start_date, end=end_date)
SOX.columns = ['SOXX_Open','SOXX_High','SOXX_Low','SOXX_Close','SOXX_Adj Close','SOXX_Volume']
SOX = SOX.drop(['SOXX_Open','SOXX_High','SOXX_Low','SOXX_Adj Close','SOXX_Volume'], axis=1)

# 미국 국채 수익률 (5년)
treasury_5y_ = yf.download("^FVX", start=start_date, end=end_date)
treasury_5y_.columns = ['treasury_5y_Open','treasury_5y_High','treasury_5y_Low','treasury_5y_Close','treasury_5y_Adj Close','treasury_5y_Volume']
treasury_5y_ = treasury_5y_.drop(['treasury_5y_Open','treasury_5y_High','treasury_5y_Low','treasury_5y_Adj Close','treasury_5y_Volume'], axis=1)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
model_samsung['exchange_rate_usd'] = exchange_rate_usd_
model_samsung['nasdaq'] = nasdaq_
model_samsung['russell_2000'] = russell_2000_
model_samsung['MSCI_KR'] = MSCI_KR
model_samsung['SOX'] = SOX
model_samsung['treasury_5y'] = treasury_5y_



In [9]:
# samsung 모델
lst_label = ['a']
for i in range(len(model_samsung)-1):
    if model_samsung.iloc[i+1]['Close'] == model_samsung.iloc[i]['Close']:
        lst_label.append(2) # 전일 주가 = 당일 주가 : 2
    elif model_samsung.iloc[i+1]['Close'] > model_samsung.iloc[i]['Close']:
        lst_label.append(0) # 전일 주가 < 당일 주가 : 0
    else:
        lst_label.append(1) # 전일 주가 > 당일 주가 : 1
model_samsung['Labeling'] = lst_label
model_samsung = model_samsung.drop(model_samsung[model_samsung['Labeling'] == 'a'].index) # 첫 행 삭제
model_samsung = model_samsung.drop(model_samsung[model_samsung['Labeling'] == 2].index) # 전일 주가 = 당일 주가인 행 삭제
model_samsung['Labeling'] = model_samsung['Labeling'].astype("category")
model_samsung = model_samsung.dropna() # 결측치가 있는 행 제거

In [10]:
model_samsung

,Open,High,Low,Close,Volume,macd,macdsignal9,macdhist,ubb,mbb,...,ma_30,ma_60,ma_120,exchange_rate_usd,nasdaq,russell_2000,MSCI_KR,SOX,treasury_5y,Labeling
날짜,,,,,,,,,,,,,,,,,,,,,
2017-06-28,47600,48000,47560,47700,191450,867.471776,726.020122,141.451654,48827.902595,46364.0,...,46040.000000,44506.666667,41783.500000,1138.93,6234.410156,1425.270020,13425,1067.180054,1.816,1
2017-06-29,48040,48320,47940,47940,166131,860.218544,752.859807,107.358737,48956.805628,46526.0,...,46093.333333,44606.000000,41882.166667,1147.30,6144.350098,1416.199951,13527,1040.420044,1.850,0
2017-06-30,47500,47620,47100,47540,237551,812.823920,764.852630,47.971291,48982.247933,46669.0,...,46146.666667,44711.666667,41974.333333,1145.43,6140.419922,1415.359985,13487,1034.910034,1.885,1
2017-07-05,46820,47680,46780,47580,212805,632.807912,718.370880,-85.562968,49091.367670,46899.0,...,46382.000000,44980.666667,42256.666667,1153.34,6150.859863,1420.150024,13430,1041.780029,1.925,0
2017-07-06,48000,48100,47720,48060,218461,643.711980,703.439100,-59.727120,49138.549116,47044.0,...,46488.000000,45084.333333,42347.000000,1157.26,6089.459961,1400.810059,13517,1037.050049,1.942,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-29,78800,78900,77300,78300,30339449,2894.138437,2674.162879,219.975558,78556.411217,73250.0,...,70983.333333,65530.000000,61147.500000,1090.97,12850.219727,1959.359985,18650,2738.050049,0.378,1
2020-12-30,77400,81300,77300,81000,29417421,3169.915677,2773.313439,396.602238,79609.789470,73910.0,...,71493.333333,65901.666667,61377.500000,1087.62,12870.000000,1979.989990,19005,2783.770020,0.370,0
2021-01-04,81000,84400,80200,83000,38655276,3509.400231,2920.530797,588.869434,81165.357133,74585.0,...,72100.000000,66301.666667,61627.500000,1086.48,12698.450195,1945.910034,19435,2783.209961,0.354,0


In [11]:
from sklearn.preprocessing import StandardScaler

from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

In [12]:
#4 데이터 정규화하기 
sc = StandardScaler()
df_scaled = pd.DataFrame(sc.fit_transform(model_samsung), columns=model_samsung.columns, index=model_samsung.index)

#4 Y 값 넣기
df_scaled.drop(['Open', 'High', 'Low', 'Volume', 'Labeling'], axis=1, inplace=True)
df_scaled['Labeling'] = model_samsung['Labeling']
df_scaled = df_scaled[1:]
df_scaled

,Close,macd,macdsignal9,macdhist,ubb,mbb,lbb,ma_5,ma_10,ma_15,...,ma_30,ma_60,ma_120,exchange_rate_usd,nasdaq,russell_2000,MSCI_KR,SOX,treasury_5y,Labeling
날짜,,,,,,,,,,,,,,,,,,,,,
2017-06-29,-0.350321,0.695199,0.647264,0.339118,-0.590540,-0.583855,-0.548801,-0.345405,-0.410494,-0.518668,...,-0.669010,-0.990339,-1.673768,0.050536,-1.330679,-0.879937,-0.000306,-1.301279,0.095534,0
2017-06-30,-0.409795,0.645775,0.660961,0.137549,-0.586731,-0.559790,-0.501055,-0.347859,-0.379338,-0.502970,...,-0.659517,-0.968901,-1.651618,0.010330,-1.333192,-0.885978,-0.031708,-1.315170,0.134160,1
2017-07-05,-0.403848,0.458048,0.607874,-0.315684,-0.570392,-0.521084,-0.436758,-0.423916,-0.385379,-0.441267,...,-0.617625,-0.914327,-1.583769,0.180400,-1.326516,-0.851529,-0.076456,-1.297850,0.178304,0
2017-07-06,-0.332479,0.469419,0.590820,-0.227994,-0.563327,-0.496683,-0.392263,-0.420236,-0.383789,-0.415322,...,-0.598756,-0.893295,-1.562061,0.264682,-1.365779,-0.990616,-0.008157,-1.309775,0.197065,0
2017-07-07,-0.362216,0.454040,0.573809,-0.227492,-0.552880,-0.481874,-0.372797,-0.410422,-0.379974,-0.390467,...,-0.585821,-0.872128,-1.540793,0.189000,-1.325097,-0.882526,-0.088231,-1.264772,0.213619,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-29,4.163745,2.816242,2.841585,0.721355,3.841498,3.913406,3.821292,3.983722,3.918515,3.952243,...,3.761624,3.254681,2.955989,-1.160593,2.957469,3.026295,4.021475,2.978740,-1.528967,1
2020-12-30,4.565194,3.103832,2.954825,1.320850,3.999223,4.024475,3.870149,4.250536,4.032963,4.053627,...,3.852408,3.330084,3.011262,-1.232620,2.970118,3.174659,4.300166,3.094008,-1.537796,0
2021-01-04,4.862563,3.457858,3.122962,1.973431,4.232143,4.138067,3.832480,4.529617,4.179203,4.152830,...,3.960400,3.411235,3.071341,-1.257131,2.860418,2.929567,4.637735,3.092596,-1.555454,0


In [13]:
#4 test, train 나누기
train_set = df_scaled[:'2020-01-01'].values
#test_set = df_scaled['2018-01-01':].values

print(df_scaled.shape)
print(train_set.shape)
#print(test_set.shape)

(791, 21)
(564, 21)


In [14]:
training_data_len = len(train_set)

In [15]:
y_columns = []
y_columns.append('samsung_change')
x_columns = df_scaled.columns.tolist()

In [16]:
# 5일치 데이터로 예측
time_seq = 5
X_train = []
y_train = []
for i in range(time_seq, train_set.shape[0]):
    X_train.append(train_set[i-5:i, 0:train_set.shape[1]-1])
    y_train.append(train_set[i, train_set.shape[1]-1:])
X_train, y_train = np.array(X_train), np.array(y_train)

In [17]:
print(X_train.shape)
print(y_train.shape)

(559, 5, 20)
(559, 1)


In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional

In [19]:
#7 딥러닝에 적용하기

# LSTM 모델 만들기
model = Sequential()
# LSTM 레이어를 쌓아올릴 때는 return_sequences 를 True로 설정한다
# LSTM 을 사용하기 위해서는 3차원 데이터가 필요(batch_size, timesteps, input_dim)

model = Sequential()
model.add(LSTM(60, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(30, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=300)

Epoch 1/300
12/12 [==============================] - 0s 7ms/step - loss: 0.6907 - accuracy: 0.5438
Epoch 2/300
12/12 [==============================] - 0s 6ms/step - loss: 0.6882 - accuracy: 0.5403
Epoch 3/300
12/12 [==============================] - ETA: 0s - loss: 0.6879 - accuracy: 0.53 - 0s 7ms/step - loss: 0.6880 - accuracy: 0.5403
Epoch 4/300
12/12 [==============================] - 0s 6ms/step - loss: 0.6849 - accuracy: 0.5456
Epoch 5/300
12/12 [==============================] - 0s 6ms/step - loss: 0.6850 - accuracy: 0.5564
Epoch 6/300
12/12 [==============================] - 0s 7ms/step - loss: 0.6825 - accuracy: 0.5528
Epoch 7/300
12/12 [==============================] - 0s 6ms/step - loss: 0.6822 - accuracy: 0.5546
Epoch 8/300
12/12 [==============================] - 0s 7ms/step - loss: 0.6832 - accuracy: 0.5635
Epoch 9/300
12/12 [==============================] - 0s 7ms/step - loss: 0.6787 - accuracy: 0.5671
Epoch 10/300
12/12 [==============================] - 0s 7ms/step -

12/12 [==============================] - 0s 7ms/step - loss: 0.3916 - accuracy: 0.8014
Epoch 164/300
12/12 [==============================] - 0s 6ms/step - loss: 0.3799 - accuracy: 0.8157
Epoch 165/300
12/12 [==============================] - 0s 6ms/step - loss: 0.3711 - accuracy: 0.8283
Epoch 166/300
12/12 [==============================] - 0s 8ms/step - loss: 0.3756 - accuracy: 0.8157
Epoch 167/300
12/12 [==============================] - 0s 6ms/step - loss: 0.3725 - accuracy: 0.8193
Epoch 168/300
12/12 [==============================] - 0s 6ms/step - loss: 0.3759 - accuracy: 0.8122
Epoch 169/300
12/12 [==============================] - 0s 7ms/step - loss: 0.3828 - accuracy: 0.8211
Epoch 170/300
12/12 [==============================] - 0s 6ms/step - loss: 0.3756 - accuracy: 0.8157
Epoch 171/300
12/12 [==============================] - 0s 7ms/step - loss: 0.3822 - accuracy: 0.8122
Epoch 172/300
12/12 [==============================] - 0s 6ms/step - loss: 0.3645 - accuracy: 0.8229
Epoc

In [20]:
# 5일치 데이터로 예측
X_test = []
y_test = []
for i in range(train_set.shape[0], df_scaled.shape[0]):
    X_test.append(df_scaled.values[i-5:i, 0:train_set.shape[1]-1])
    y_test.append(df_scaled.values[i, train_set.shape[1]-1:])
X_test, y_test = np.array(X_test), np.array(y_test)

predictions = model.predict(X_test)

In [21]:
eval = model.evaluate(X_test, (y_test))
print('{0} 정확도: {1:.4f}, Loss: {2:.4f}'.format(model.__class__.__name__, eval[1], eval[0]))

8/8 [==============================] - 0s 2ms/step - loss: 2.1227 - accuracy: 0.5066
Sequential 정확도: 0.5066, Loss: 2.1227


In [22]:

# 0.5 기준으로 0, 1 분류
def tmp(x) :
    if x > 0.5 :
        return 1
    else :
        return 0
pred_bin = np.vectorize(tmp)

pred = pred_bin(predictions)
pred

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [23]:
# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse

0.6666223465695705